<a href="https://colab.research.google.com/github/tricksterwink/EarthQuakePrediction/blob/main/Earthquake_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Earth Quake Prediction** based on acoustic data, using a linear regression model

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


# Importing Libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import NuSVR, SVR
from sklearn.kernel_ridge import KernelRidge
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn import tree
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
import statsmodels.api as sm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn import neighbors
from sklearn import linear_model

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.2 MB/s 


In [ ]:
train=pd.read_csv("/content/drive/MyDrive/Earthquake Detection/LANL-Earthquake-Prediction/train.csv")

In [ ]:
def gen_features(X):
    strain = []
    strain.append(X.mean())
    strain.append(X.std())
    strain.append(X.min())
    strain.append(X.max())
    strain.append(X.kurtosis())
    strain.append(X.skew())
    strain.append(np.quantile(X,0.01))
    strain.append(np.quantile(X,0.05))
    strain.append(np.quantile(X,0.95))
    strain.append(np.quantile(X,0.99))
    strain.append(np.abs(X).max())
    strain.append(np.abs(X).mean())
    strain.append(np.abs(X).std())
    return pd.Series(strain)

In [ ]:
X = train.drop(['time_to_failure'], axis=1)
y = train.drop(['acoustic_data'], axis=1)

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.7, random_state=1234) 

In [ ]:
from sklearn import datasets, linear_model

regr = linear_model.LinearRegression()

regr.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_predicted = regr.predict(X_validation)
MAE = mean_absolute_error(y_validation, y_predicted)
print(MAE)

0.0680679847426949


In [ ]:
import os

train_folder = '/content/drive/MyDrive/Earthquake Detection/LANL-Earthquake-Prediction/test'

files_in_train = sorted(os.listdir(train_folder))

files=[i for i in files_in_train ]

seg_id=[]
time_to_failure=[]

for i in files:

  test =pd.read_csv("/content/drive/MyDrive/Earthquake Detection/LANL-Earthquake-Prediction/test/" + i)

  submission = pd.DataFrame()
  submission['acoustic_data'] = test['acoustic_data']
  submission['time_to_failure'] = regr.predict(test)
  z=submission['time_to_failure'].mean()

  seg_id.append(i)
  time_to_failure.append(z)

seg_id=np.array(seg_id)
time_to_failure=np.array(time_to_failure)

df = pd.DataFrame({'seg_id':seg_id, 'time_to_failure':time_to_failure})

df.to_csv("final.csv")

In [ ]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]
  
from catboost import CatBoostRegressor
model=CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.1, loss_function='MAE', silent =True)
model.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation), plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
y_predicted = model.predict(X_validation)
MAE = mean_absolute_error(y_validation, y_predicted)
print(MAE)

0.06806067386753412


In [ ]:
model=linear_model.Ridge()
model.fit(X_train,y_train)

Ridge()

In [ ]:
y_predicted = model.predict(X_validation)
MAE = mean_absolute_error(y_validation, y_predicted)
print(MAE)

0.06806798474271736


In [ ]:
n_neighbors=5
knn=neighbors.KNeighborsRegressor(n_neighbors,weights='uniform')
knn.fit(X_train,y_train)

KNeighborsRegressor()

In [ ]:
y_predicted = model.predict(X_validation)
MAE = mean_absolute_error(y_validation, y_predicted)
print(MAE)

0.06806798474271736


In [ ]:
reg = linear_model.BayesianRidge()
reg.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BayesianRidge()

In [ ]:
y_predicted = model.predict(X_validation)
MAE = mean_absolute_error(y_validation, y_predicted)
print(MAE)

0.06806798474271736


In [ ]:
dec = tree.DecisionTreeRegressor(max_depth=3)
dec.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=3)

In [ ]:
y_predicted = model.predict(X_validation)
MAE = mean_absolute_error(y_validation, y_predicted)
print(MAE)

0.06806798474271736


In [ ]:
from sklearn.linear_model import LinearRegression, RANSACRegressor

ransac = RANSACRegressor(base_estimator=LinearRegression(),
                         min_samples=50, max_trials=100,
                         loss='absolute_loss', random_state=42,
                         residual_threshold=10)

ransac.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ransac.py:358: FutureWarning: The loss 'absolute_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='absolute_error'` which is equivalent.
  FutureWarning,


RANSACRegressor(base_estimator=LinearRegression(), loss='absolute_loss',
                min_samples=50, random_state=42, residual_threshold=10)

In [ ]:
y_predicted = model.predict(X_validation)
MAE = mean_absolute_error(y_validation, y_predicted)
print(MAE)

0.06806798474271736
